In [1]:
import torch
import json

from transformers import AutoTokenizer
from transformers.models.bloom.configuration_bloom import BloomConfig
from pruning.pruned_bloom import PrunedBloomForCausalLM

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
weights_path = "pruned_560m_bloom.pt"
state_dict_shapes_path = "state_dict_shapes.pkl"
config_path = "bloom_560m_config.json"
config_json = json.load(open(config_path, "rb"))
bloom_config = BloomConfig(
    vocab_size=250880,
    hidden_size=1024,
    n_layer=24,
    n_head=16,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    use_cache=True,
    bos_token_id=1,
    eos_token_id=2,
    apply_residual_connection_post_layernorm=False,
    hidden_dropout=0.0,
    attention_dropout=0.0,
    pretraining_tp=1,  # TP rank used when training with megatron
    slow_but_exact=False,
)

In [3]:
pruned_model = PrunedBloomForCausalLM(bloom_config, state_dict_shapes_path)

In [4]:
pruned_model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [5]:
tokenizer = AutoTokenizer.from_pretrained(f"bigscience/bloom-560m")

In [6]:
line = "Hello, I am an AlexPrize chatbot"
inputs = tokenizer(line, return_tensors="pt")
outputs = pruned_model.generate(
    input_ids=inputs["input_ids"], 
    max_new_tokens=1, 
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    return_dict_in_generate=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: The size of tensor a (1024) must match the size of tensor b (801) at non-singleton dimension 2